In [ ]:
import sys
import os
import warnings
import random
import gradio as gr
import model_loader

# Core libraries
import torch
import numpy as np
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from transformers import CLIPTokenizer

DEVICE = "cpu"

ALLOW_CUDA = False
ALLOW_MPS = False

if torch.cuda.is_available() and ALLOW_CUDA:
    DEVICE = "cuda"
elif (torch.has_mps or torch.backends.mps.is_available()) and ALLOW_MPS:
    DEVICE = "mps"
print(f"Using device: {DEVICE}")


In [ ]:
# Load models and tokenizer
tokenizer = CLIPTokenizer("data/vocab.json", merges_file="data/merges.txt")
model_file = "data/v1-5-pruned-emaonly.ckpt"
models = model_loader.preload_models_from_standard_weights(model_file, DEVICE)
print("Models loaded successfully!")

## 🔧 Core Inpainting Function

The main function that powers our web interface:

In [ ]:
import inpainting

def run_inpainting(image, mask, prompt, negative_prompt, strength, cfg_scale, num_steps, seed):
    if image is None or mask is None:
        raise gr.Error("Please upload both image and mask")
    if not prompt.strip():
        raise gr.Error("Prompt is required")

    # Resize mask to match image
    mask = mask.convert("L").resize(image.size)
    mask_np = np.array(mask)
    original_size = image.size  

    if seed == -1:
        seed = random.randint(0, 999999)

    result = inpainting.inpaint(
        prompt=prompt,
        image=image,
        mask=mask_np,
        uncond_prompt=negative_prompt,
        strength=strength,
        do_cfg=True,
        cfg_scale=cfg_scale,
        sampler_name="ddpm",
        n_inference_steps=num_steps,
        models=models,
        tokenizer=tokenizer,
        seed=seed,
        device=DEVICE,
        idle_device="cpu"
    )

    result_image = Image.fromarray(result)
    result_image = result_image.resize(original_size, resample=Image.LANCZOS)

    return result_image


## 🚀 Launch Web Interface

Start the web server and access your inpainting studio:

In [ ]:
with gr.Blocks(title="Stable Diffusion Inpainting", theme=gr.themes.Soft()) as demo:
    gr.Markdown("## Inpainting (Gradio 5.41.1)")

    with gr.Row():
        with gr.Column():
            image_input = gr.Image(label="Upload Your Image", type="pil")

            mask_input = gr.Image(
                label="Draw Mask (white = inpaint)",
                type="pil"
            )

        with gr.Column():
            prompt = gr.Textbox(label="📝 Prompt", lines=2, placeholder="e.g. a mountain with a castle")
            negative_prompt = gr.Textbox(label="🚫 Negative Prompt", value="blurry, low quality", lines=1)
            strength = gr.Slider(0.1, 1.0, step=0.1, value=0.8, label="Strength")
            cfg_scale = gr.Slider(1.0, 20.0, step=0.5, value=7.5, label="CFG Scale")
            num_steps = gr.Slider(10, 100, step=5   , value=30, label="Denoising Steps")
            seed = gr.Number(value=-1, precision=0, label="Seed (-1 = random)")
            run_button = gr.Button("🚀 Generate")

    output_image = gr.Image(label="🖼️ Output Image")

    run_button.click(
        fn=run_inpainting,
        inputs=[image_input, mask_input, prompt, negative_prompt, strength, cfg_scale, num_steps, seed],
        outputs=output_image
    )

demo.launch()
